## Setup

In [2]:
import torch
torch.cuda.set_device(5)

from slice_e2e_test import *
pd.options.display.max_rows = 999

In [3]:
df_sids = pd.DataFrame(sids).set_index(0)
df_slices = df_sids.merge(df, left_index=True, right_index=True).reset_index().set_index('SOPInstanceUID')

In [4]:
dl = dbunch.valid_dl
osids = L(dl).itemgot(0).map(list)
ssids = L(sum(osids,[]))
df2 = pd.DataFrame(ssids).set_index(0)

In [27]:
pres = '''train_rn18_all-e2e rn18_wgtd_freeze2-e2e train_rn18-e2e
    rn18_wgtd-e2e-fr train_rn34-e2e-fr xrn34_wgtd_deep-e2e-fr'''.split()

all_preds = [(path_pred_test/f'{pre}.pkl'     ).load() for pre in pres]
flp_preds = [(path_pred_test/f'{pre}-flip.pkl').load() for pre in pres]

In [65]:
avg_pred = torch.stack(all_preds+flp_preds).mean(0).sigmoid()

In [31]:
a=1e-7
avg_pred = torch.stack(all_preds+flp_preds).mean(0).sigmoid().clamp(a,1-a)

In [33]:
dfc = submission(df2, avg_pred, fn='noclamp-final')